In [1]:
import os
import sys
import numpy as np

sys.path.append("./resources")
from resources.utils import scrape_html, to_word_list
from resources.keywords import extract_keywords

**EXAMPLE FOR GETTING TEXT FROM FILE**

In [2]:
corpus_path = os.path.join(os.getcwd(), "corpus")
filepath = os.path.join(corpus_path, os.listdir(corpus_path)[0])
print(filepath)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

sent_list = scrape_html(filepath)

res = sent_list[0]

for sent in sent_list[1:]:
    if res[-1] in u"[{(:;'.,\"01234567890אבגדהוזחטיכלמנסעפצקרשתףץםןך"\
            and sent[0] not in ")}]\n":
        res += " " + sent
    else:
        res += sent

print(res)

g:\האחסון שלי\Colab Notebooks\Natural_Language_Processing_10247\Homework\nlp_final_proj\corpus\21063160.N26.html
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
בבית המשפט העליון בשבתו כבית משפט
  גבוה לצדק בג"ץ 6316/21 לפני: כבוד השופט ד' מינץ כבוד השופט י' אלרון כבוד השופט י' כשר העותרת: אגודת מגן דוד אדום בישראל נ ג ד המשיבים: 1. שר התקשורת 2.
  הועדה הבין-משרדית להקצאת מספרים מקוצרים למוקדי חרום, מצוקה ורווחה 3.
  שר הבריאות 4.
  מנכ"ל משרד הבריאות 5.
  משרד הבריאות 6.
  איחוד הצלה ישראל 1221 עתירה למתן צו על תנאי תאריך הישיבה: כ"ז בחשוון התשפ"ג (21.11.2022)
  בשם העותרת: עו"ד
  יעל ארידור בר-אילן; עו"ד רונית מאיר וילף בשם משיבים 5-1: עו"ד אודי איתן בשם משיב 6: עו"ד עמית חדד; עו"ד נועה
  מילשטיין פסק-דין השופט ד' מינץ: בעתירה
שלפנינו התבקש ביטול המספר המקוצר שהוקצה על ידי משרד התקשורת למשיבה 6 (להלן: איחוד הצלה). 1. ברקע לעתירה ובבסיס לה עומדת החלטה משנת 2010 להקצות מספר טלפון מקוצר
לאיחוד הצלה (1221). הקצאה זו נעשתה בעקבות המלצ

In [3]:
docs = {}

for filename in os.listdir(corpus_path):
    filepath = os.path.join(corpus_path, filename)
    doc = " ".join((" ".join(scrape_html(filepath))).split())
    
    docs[filename] = to_word_list(doc)

In [4]:
# res = extract_keywords(docs, 'word2vec')

In [5]:
# display(res)

In [6]:
raw_docs = {}

for filename in os.listdir(corpus_path):
    filepath = os.path.join(corpus_path, filename)
    raw_docs[filename] = scrape_html(filepath)
    raw_docs[filename] = [" ".join(s.split()) for s in raw_docs[filename]]

In [7]:
from resources.sentiment import sentiment_tag

sent_docs = sentiment_tag(raw_docs)

In [10]:
from resources.ner import extract_ner
import pandas as pd

ner_docs, tagged_sents = extract_ner(raw_docs)